# EC1B1 Macroeconomics Coursework - Code

This notebook contains all the code for our EC1B1 Macroeconomics coursework. The code is divided into the following sections:

## 0. Import Libraries

In [28]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

## 1. Load and Preprocess Data

We first load the Italy and USA data into separate dataframes.

In [29]:
italy_df = pd.read_excel('data/IMF_Italy.xlsx', header=1, index_col=0)
usa_df = pd.read_excel('data/IMF_US.xlsx', header=1, index_col=0)

### 1.1 Inspect Data

In [30]:
italy_df.head()

,"Economic Activity, Industrial Production, Index","Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate","International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar","Prices, Consumer Price Index, All items, Index"
Jan 1960,31.134278,621.118012,2989.5,4.143942
Feb 1960,32.602650,621.118012,2958.5,4.136754
Mar 1960,33.712883,620.600000,2923.5,4.129566
Apr 1960,33.891953,625.000001,2959.5,4.125971
May 1960,34.632109,625.000001,2970.5,4.133160


In [31]:
usa_df.head()

,"International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar","Prices, Consumer Price Index, All items, Index"
Jan 1960,21478.1,13.436946
Feb 1960,21395.7,13.482806
Mar 1960,21344.7,13.482806
Apr 1960,21278.0,13.528666
May 1960,21234.3,13.528666


We will shorten the column names for easier inspection and reference:

In [32]:
italy_df.columns = ['industrial_production_index', 'nominal_er_nat_currency_per_usd', 'reserves_in_usd', 'cpi']
usa_df.columns = ['reserves_in_usd', 'cpi_index']

In [33]:
italy_df.head()

,industrial_production_index,nominal_er_nat_currency_per_usd,reserves_in_usd,cpi_index
Jan 1960,31.134278,621.118012,2989.5,4.143942
Feb 1960,32.602650,621.118012,2958.5,4.136754
Mar 1960,33.712883,620.600000,2923.5,4.129566
Apr 1960,33.891953,625.000001,2959.5,4.125971
May 1960,34.632109,625.000001,2970.5,4.133160


In [34]:
usa_df.head()

,reserves_in_usd,cpi_index
Jan 1960,21478.1,13.436946
Feb 1960,21395.7,13.482806
Mar 1960,21344.7,13.482806
Apr 1960,21278.0,13.528666
May 1960,21234.3,13.528666


### 1.2 Merge Data

In [35]:
df = pd.merge(italy_df, usa_df, left_index=True, right_index=True, suffixes=('_italy', '_usa'))
df.head()

,industrial_production_index,nominal_er_nat_currency_per_usd,reserves_in_usd_italy,cpi_index_italy,reserves_in_usd_usa,cpi_index_usa
Jan 1960,31.134278,621.118012,2989.5,4.143942,21478.1,13.436946
Feb 1960,32.602650,621.118012,2958.5,4.136754,21395.7,13.482806
Mar 1960,33.712883,620.600000,2923.5,4.129566,21344.7,13.482806
Apr 1960,33.891953,625.000001,2959.5,4.125971,21278.0,13.528666
May 1960,34.632109,625.000001,2970.5,4.133160,21234.3,13.528666


## 2. Identify Outliers

## 3. Construct Variables

We will now construct the following variables as per the coursework instructions:

1. The monthly growth in the nominal exchange rate
2. The real exchange rate
3. The monthly growth in the real exchange rate
4. The monthly inflation rate
5. The monthly growth in industrial production
6. The growth in industrial production versus 12 months ago (i.e. January 1971 versus January 1970 etc.)
7. An index of the value of international reserves (value of reserves at January 1960 = 100)

### 3.1 Monthly Nominal Exchange Rate Growth

### 3.2 Real Exchange Rate

The real exchange rate is calculated as the nominal exchange rate multiplied by the ratio of the price index in Italy to the price index in the USA.

$$ ER_{real} = ER_{nominal} \times \frac{P_{Italy}}{P_{US}} $$

In [36]:
df.insert(df.columns.get_loc('nominal_er_nat_currency_per_usd') + 1,
          'real_er_nat_currency_per_usd',
          df['nominal_er_nat_currency_per_usd'] * df['cpi_index_italy'] / df['cpi_index_usa'])

In [37]:
df

,industrial_production_index,nominal_er_nat_currency_per_usd,real_er_nat_currency_per_usd,reserves_in_usd_italy,cpi_index_italy,reserves_in_usd_usa,cpi_index_usa
Jan 1960,31.134278,621.118012,191.552239,2989.500000,4.143942,21478.100000,13.436946
Feb 1960,32.602650,621.118012,190.569544,2958.500000,4.136754,21395.700000,13.482806
Mar 1960,33.712883,620.600000,190.079779,2923.500000,4.129566,21344.700000,13.482806
Apr 1960,33.891953,625.000001,190.612448,2959.500000,4.125971,21278.000000,13.528666
May 1960,34.632109,625.000001,190.944545,2970.500000,4.133160,21234.300000,13.528666
...,...,...,...,...,...,...,...
Aug 1990,51.945660,1158.680000,1078.009512,94282.568583,56.149763,78908.838357,60.351608
Sep 1990,105.180751,1171.700000,1086.797941,92443.529313,56.446401,80024.166133,60.856066
Oct 1990,115.588303,1141.680000,1061.299970,92157.185028,56.912541,82852.196532,61.222946
Nov 1990,106.193875,1116.310000,1041.557226,87764.009459,57.251568,83059.402774,61.360525


### 3.3 Monthly Real Exchange Rate Growth

### 3.4 Monthly Inflation Rate (Italy and USA)

In [ ]:
df.insert(df.columns.get_loc('cpi_usa') + 1,
          'inflation_rate_usa',
          df['nominal_er_nat_currency_per_usd'] * df['cpi_index_italy'] / df['cpi_index_usa'])

$$ 1+1=2